In [1]:
import ee, math, os, sys

import geemap

ee.Initialize()

from utils.Masks import Masks
from utils.Filters.CorrectionLIA import CorrectionLIA
from utils.UtilsSAR import *
from utils.Filters.QueganYuFilter import QueganYuFilter
from utils.Filters.LeeFilter import LeeFilterRefined

## Config Information
### Static variables, assets, periods, etc

In [2]:
# periods
DETECTION_PERIOD = {
    't0': '2023-01-01',
    't1': '2023-01-29'
}

LEARNING_DATE_START =  {
    't0': '2022-01-01',
    't1': '2022-01-29'
}

CHECK_BREAKS = {
    't0': '2022-06-01',
    't1': '2022-10-30'
}


ASSET_DETER = 'users/jailson/inpe/deter-amz-20-08-2023-14_26_52'
ASSET_FOREST_MASK = ''
ASSET_DEFORESTATION_MASK = ''
ASSET_ROI = 'projects/ee-artigo/assets/APA_TX'

TARGET_BAND = 'VVg0'

Map = geemap.Map()


## Input Data

In [3]:

alertsDETER = ee.FeatureCollection(ASSET_DETER).map(
    lambda feat: feat
        .set('system:time_start', feat.get('VIEW_DATE'))
        .set('system:time_end', feat.get('VIEW_DATE'))
)

alertsDETER = alertsDETER.filterDate(CHECK_BREAKS['t0'], CHECK_BREAKS['t1'])


roi = ee.FeatureCollection(ASSET_ROI)

collectionS1 = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')\
    .filterBounds(roi)\
    .filterDate(LEARNING_DATE_START['t0'], DETECTION_PERIOD['t1'])\
    .filter(ee.Filter.And(
        ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'),
        ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'),
        ee.Filter.eq('instrumentMode', 'IW')
    )) \
    .select('VV','VH','angle') 



## Check Input Data
### Number of images, polygons, groups by metadata, etc

In [4]:
# 
#countImages = collectionS1.limit(1).getInfo()
#countImages

## Helper Functions
### Auxiliary functions 

In [5]:
def toGamma0natural(img):

    azimuthEdge = getDESCCorners(img)
    
    lia = CorrectionLIA(img, azimuthEdge)
    lia = lia.getLIA()
    
    # WARNING: INPUT VALUES MUST BE IN linear scale!!!! BAND0: VV BAND1:VH
    img = img.addBands(lia.rename('LIA')) # Computes Local Incidence Angle (LIA) band
    lia = img.select('LIA')
    vv_gamma0 = img.select(0).divide(lia.multiply(math.pi/180.0).cos())
    vh_gamma0 = img.select(1).divide(lia.multiply(math.pi/180.0).cos())

    return img.addBands(vv_gamma0.rename('VVg0')) \
        .addBands(vh_gamma0.rename('VHg0')) \
        .addBands(lia.rename('LIA'))



def computeLogisticThreshold(mask, output_scale, lim_min, lim_max):
  # Logistic distribution parameters extracted from PRODES distance to previous year analysis
  mu = 2.40633  
  s = 0.29973
  max_pdf = 1 / (4*s)
  
  dist = mask.fastDistanceTransform().sqrt().multiply(output_scale)
  
  threshold = getLogisticDistPdf(dist.log10(), mu, s)\
          .where(dist.log10().lt(mu),max_pdf)\
          .multiply((lim_min-lim_max)/max_pdf).add(lim_max)\
          .unmask(lim_min,False)   
  
  return threshold

def getLogisticDistPdf(x,mu,s):
  x= ee.Image(x) 
  mu = ee.Image(mu) 
  s = ee.Image(s)
  normx = x.subtract(mu).divide(s.multiply(2))
  return normx.cosh().pow(-2).multiply(s.multiply(4).pow(-1))



## PreProcessing

In [6]:
masks = Masks()

# apply masks onto collection and reescale numbers
collectionS1_ = ee.ImageCollection(
    collectionS1.sort('system:time_start') 
    .map(masks.getAngleMask)
    .map(masks.getMaskBorders)
    .map(toGamma0natural)
).select(["VHg0", "VVg0", "LIA"])




# apply filters
# 1. Quegan&Yu (2001) Filter
queganYuFilter = QueganYuFilter(collectionS1_, kernel_size=5)
collectionS1F = queganYuFilter.apply()

# 2. Lee Filter 
collectionS1F_ = collectionS1F.select(TARGET_BAND).map(LeeFilterRefined.apply)


colS1F2 = collectionS1F_.combine(collectionS1_.select("LIA"))

colS1F2 = colS1F2.map(lambda img: img
        .updateMask(img.select("LIA").gt(28))
        .updateMask(img.select("LIA").lt(50))) \
        .select(0)

colS1F2 = ee.ImageCollection(colS1F2)



## Detection Method

In [7]:

learnCol = collectionS1F_.select(0).filterDate(LEARNING_DATE_START['t0'], LEARNING_DATE_START['t1'])

detectionCol = colS1F2.select(0).map(toDB).filterDate(DETECTION_PERIOD['t0'], DETECTION_PERIOD['t1'])



## Display images

In [8]:
# vis params
vis = {
    'min':-20,
    'max':20,
    'palette': ['red', 'gray', 'white', 'black']
}

Map.addLayer(detectionCol, vis)

Map.addLayer(alertsDETER)

Map.setOptions('SATELLITE')

Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…